In [ ]:
import pandas as pd
import re
import pandas as pd
import nltk
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('omw-1.4')
nltk.download('words')
from textblob import TextBlob
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
nltk.download('stopwords')
pos=[]
neg=[]
total_sen=[]
tweets=[]

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!
[nltk_data] Downloading package words to /root/nltk_data...
[nltk_data]   Package words is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [ ]:
!sudo apt-get install -y swig3.0
!pip install jamspell
!wget -nc https://github.com/bakwc/JamSpell-models/raw/master/en.tar.gz
!tar -xf en.tar.gz

Reading package lists... Done
Building dependency tree       
Reading state information... Done
swig3.0 is already the newest version (3.0.12-1).
The following package was automatically installed and is no longer required:
  libnvidia-common-460
Use 'sudo apt autoremove' to remove it.
0 upgraded, 0 newly installed, 0 to remove and 12 not upgraded.
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
File ‘en.tar.gz’ already there; not retrieving.



In [ ]:
import jamspell
corrector = jamspell.TSpellCorrector()
corrector.LoadLangModel('en.bin')

True

In [ ]:
from nltk.corpus import words
wordlist=words.words()


In [ ]:
dataset = pd.read_csv("A1_dataset.csv")
text=""
text_l=[]
columns_n = ['LABEL','DATE_TIME','TEXT']
processed_dataset = pd.DataFrame(columns=columns_n)
c=0

for i,d in dataset.iterrows():
  c+=1
  # print(c)
  text = d["TEXT"]
  text = text.lower().strip()
  text = re.sub(r"\<\w+>",'',text)
  text = re.sub(r"\bwww.\w+.\w+/?\w*",'',text)
  text = re.sub(r"\bhttp://\w+.\w+/\w+",'',text)
  text = re.sub(r"\@\w+",'',text)
  text = re.sub(r"\#\w+",'',text)
  text = re.sub(r'\s+',' ',text) 
  text = re.sub(r'[^\w\s]','',text)
  pattern = re.compile(r'\b(' + r'|'.join(stopwords.words('english')) + r')\b\s*')
  text = pattern.sub('', text)
  text = corrector.FixFragment(text)
  text=text.strip()
  text_l= word_tokenize(text)
  buff_str=''
  for i in text_l:
    if i in wordlist:
      buff_str=buff_str+ i +' '
  text=buff_str[:]
  if text!="":
    processed_dataset = processed_dataset.append({'LABEL':d['LABEL'],'DATE_TIME':d['DATE_TIME'],'TEXT':text},ignore_index =True)
processed_dataset.to_csv('processed_data.csv')
  

In [ ]:
dataset=pd.read_csv("processed_data.csv")
for i, row in dataset.iterrows():
  tweets.append(row["TEXT"])
  if row["LABEL"]==1:
    pos.append(row["TEXT"])
  else:
    neg.append(row["TEXT"])


In [ ]:
unigram_dict={}
bigram_dict={}

for j in tweets:
  buffarr=j.split()
  for i in buffarr:
    if i in unigram_dict:
      unigram_dict[i]=unigram_dict[i]+1
    else:
      unigram_dict[i]=1
  
for i in tweets:
  for j in i.split():
    bigram_dict[j]={}

for i in tweets:
  buff_arr=i.split()
  for j in range(1, len(buff_arr)):
    if buff_arr[j-1] in bigram_dict[buff_arr[j]]:
      bigram_dict[buff_arr[j]][buff_arr[j-1]]=bigram_dict[buff_arr[j]][buff_arr[j-1]] + 1
    else:
      bigram_dict[buff_arr[j]][buff_arr[j-1]]=1

top4={}
for i in bigram_dict:
  top4[i]={}

for i in bigram_dict:
  for j in bigram_dict[i]:
    top4[i][j]=bigram_dict[i][j]

for i in bigram_dict:
  for j in unigram_dict:
    if j not in bigram_dict[i]:
      bigram_dict[i][j]=0.5
    else:
      bigram_dict[i][j]+=0.5

In [ ]:
import json
with open('convert.txt','w') as convert_file:
  convert_file.write(json.dumps(bigram_dict))

In [ ]:
!pip install vaderSentiment

In [ ]:
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
sid_obj = SentimentIntensityAnalyzer()

def sentiment_scores(sentence):
    global sid_obj
    sentiment_dict = sid_obj.polarity_scores(sentence)
    if sentiment_dict['compound'] >= 0.05:
        return [1, sentiment_dict['compound']]
    elif sentiment_dict['compound'] < 0.05 and sentiment_dict['compound'] >= 0:
        return [1, 0]
    elif sentiment_dict['compound'] > -0.05 and sentiment_dict['compound'] < 0:
      return [-1, 0]
    else:
      return [-1, -(sentiment_dict['compound'])]

bigram_prob={}
for i in bigram_dict:
  bigram_prob[i]={}

for i in bigram_dict:
  for j in bigram_dict[i]:
    bigram_prob[i][j]=bigram_dict[i][j]
c=0
v=len(unigram_dict)
for word in bigram_prob:
  # print(c)
  c+=1
  for prev in bigram_prob[word]:
    string=word+' '+prev
    sent=sentiment_scores(string)
    bigram_prob[word][prev]=((bigram_dict[word][prev])/(unigram_dict[prev] + 0.5*v))*sent[0] + sent[1]


In [ ]:
import random
keys_bigram = list(bigram_dict)
sentences = []

for i in range(250):
  sen=""
  word = random.choice(keys_bigram)
  sen = word[:]
  arr=[word]
  for j in range(10):
    sub_dict = bigram_dict[word]
    buff_dict={}
    for i in sub_dict:
      if i not in arr:
        buff_dict[i]=sub_dict[i]
    
    maxprob=max(list(buff_dict.values()))
    words=[i for i in buff_dict if buff_dict[i]==maxprob]
    prev_word=random.choice(words)
    arr.append(prev_word)
    sen= prev_word+ ' ' + sen
    word=prev_word[:]
  sentences.append(sen)

# for i in sentences:
#   print(i)

for i in range(250):
  sen=""
  word = random.choice(keys_bigram)
  sen = word[:]
  arr=[word]
  for j in range(10):
    sub_dict = bigram_dict[word]
    buff_dict={}
    for i in sub_dict:
      if i not in arr:
        buff_dict[i]=sub_dict[i]
    
    minprob=min(list(buff_dict.values()))
    words=[i for i in buff_dict if buff_dict[i]==minprob]
    prev_word=random.choice(words)
    arr.append(prev_word)
    sen= prev_word+ ' ' + sen
    word=prev_word[:]
  sentences.append(sen)


In [ ]:
def sentiment_scores1(sentence):
    global sid_obj
    sentiment_dict = sid_obj.polarity_scores(sentence)
    if sentiment_dict['compound'] >= 0.05:
        return 1
    else:
      return 0

generated_labels=[]
for i in sentences:
  label = sentiment_scores1(i)
  if label==1:
    generated_labels.append(0)
  else:
    generated_labels.append(1)
generated_data = pd.DataFrame(list(zip(sentences, generated_labels)),
               columns =['TEXT', 'LABEL'])
generated_data.to_csv("generated_data.csv")

In [ ]:
def perplexity(sentence):
  arr=sentence.split()
  prob=1
  for i in range(1, len(arr)):
    prob=abs(bigram_prob[arr[i]][arr[i-1]])*prob

  prob_inv=1/prob
  power=1/len(arr)
  perp=pow(prob_inv, power)

  return perp

In [ ]:
sum=0
# print(sentences)
for i in sentences:
  sum=sum+perplexity(i)

avg=sum/500
print(avg)

In [ ]:
bigram_df = pd.DataFrame(columns=["PREV","WORD","COUNT"])
c=0
for i in top4:
  c+=1
  # print(c)
  for j in top4[i]:
    bigram_df = bigram_df.append({"PREV":j,"WORD":i,"COUNT":bigram_dict[i][j]},ignore_index=True)
bigram_df.nlargest(4, "COUNT")

In [ ]:
count=0
print("POSITIVE SENTENCES")
for i,k in generated_data.iterrows():
  if k['LABEL']==0 and count<=5:
    print(k['TEXT'])
    count +=1
count=0
print("NEGATIVE SENTENCES")
for i,k in generated_data.iterrows():
  if k['LABEL']==1 and count<=5:
    print(k['TEXT'])
    count +=1

In [ ]:
dataA_B = pd.read_csv("A1_dataset.csv")
dataA_B = dataA_B.drop('DATE_TIME', axis=1)
for i in range(len(sentences)):
  dataA_B= dataA_B.append({"TEXT":sentences[i],"LABEL":generated_labels[i]},ignore_index=True)

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import make_pipeline
from sklearn.metrics import accuracy_score
def train_and_evaluate(train_sentences, train_labels,
  test_sentences, test_labels):
  """
    parameters:
    train_sentences : list of training sentences
    train_labels : list of training labels
    test_sentences : list of test sentences
    test_labels : list of test labels
    output:
    accuracy : accuracy of the test set
  """
  # Model building
  model = make_pipeline(TfidfVectorizer(), MultinomialNB())
  # Training the model with the training data
  model.fit(train_sentences, train_labels)
  # Predicting the test data categories
  predicted_test_labels = model.predict(test_sentences)
  return accuracy_score(test_labels, predicted_test_labels)

In [ ]:
DatasetA = pd.read_csv("A1_dataset.csv")
test_set = pd.read_csv("A2_test_dataset.csv")
train_sentences = DatasetA['TEXT'].tolist()
train_labels = DatasetA['LABEL'].tolist()
test_sentences = test_set['TEXT'].tolist()
test_labels = test_set['LABEL'].tolist()
acc_A = train_and_evaluate(train_sentences, train_labels,test_sentences, test_labels)
print(acc_A)

In [ ]:
train_sentences = dataA_B['TEXT'].tolist()
train_labels = dataA_B['LABEL'].tolist()
acc_B = train_and_evaluate(train_sentences, train_labels,test_sentences, test_labels)
print(acc_B)